In [1]:
from bs4 import BeautifulSoup
import requests, time, random
import mysql.connector
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import threading

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="violet",
  database="appinfo"
)

In [3]:
mycursor = mydb.cursor()

sql = '''CREATE TABLE IF NOT EXISTS ios_purpose (
    id int primary key auto_increment,
    app_name text,
    analytics text,
    app_functionality text,
    developer’s_advertising_or_marketing text,
    other_purposes text,
    product_personalisation text,
    thirdparty_advertising text
)
'''

mycursor.execute(sql)
mydb.commit()

data_categories = [
    "Analytics", "App Functionality", "Developer’s Advertising or Marketing", "Other Purposes", 
    "Product Personalisation", "Third-Party Advertising"
]

In [4]:
def findName(soup):
    global app_name
    for x in soup.findAll('h1',class_='product-header__title app-header__title'):
        for span in soup.findAll('span',class_='badge badge--product-title'):
            span.decompose()
        app_name = x.text
        print(app_name)
    return app_name

In [5]:
def scrape_data(url, data_categories):
    try:
        options = Options()
        options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
        options.add_argument('--headless')
        driver = webdriver.Firefox(executable_path='F:\python\geckodriver.exe', options=options)
        driver.get(url)
        
        see_details_buttons = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.XPATH, "//*[text()='See Details']"))
        )
        
        for button in see_details_buttons: 
            button.click()
            
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        app_name  = findName(soup)
        
        h3_tags = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.XPATH, "//h3[@class='privacy-type__purpose-heading']"))
        )
        
        data_categories = {category: set() for category in data_categories}
        for i in range(len(h3_tags)):
            current_element = h3_tags[i]
            category = current_element.text
            while True:
                try:
                    current_element = current_element.find_element(By.XPATH,  "following-sibling::*[1]")
                    if current_element.get_attribute("class") == "privacy-type__grid":
                        li_elements = current_element.find_elements(By.CSS_SELECTOR, "div.privacy-type__grid-content ul li")
                        for li_element in li_elements:
                            text = li_element.text
                            data_categories[category].add(text)
                    elif current_element.get_attribute("class") == "privacy-type__purpose-heading":
                        break
                except Exception as e:
                    break
                    
        mycursor = mydb.cursor()
        
        data = {}
        for category, data_set in data_categories.items():
            data[category] = ','.join(data_set)
        
        sql = f"INSERT INTO ios_purpose (app_name, analytics, app_functionality, developer’s_advertising_or_marketing, other_purposes, product_personalisation, thirdparty_advertising) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        val = (app_name, ', '.join(data_categories['Analytics']), ', '.join(data_categories['App Functionality']), ', '.join(data_categories['Developer’s Advertising or Marketing']), ', '.join(data_categories['Other Purposes']), ', '.join(data_categories['Product Personalisation']), ', '.join(data_categories['Third-Party Advertising']))
        mycursor.execute(sql, val)
        mydb.commit()
        
    except TimeoutException:
        print(f"Timeout exception occurred while loading {url}")
        
    except NoSuchElementException:
        print(f"Element not found exception occurred while loading {url}")
        
    finally:
        driver.quit()

In [6]:
with open('ios.txt','r',encoding='utf-8') as f:
    urllst = f.readlines()

threads = []
for urls in urllst:
    thread = threading.Thread(target=scrape_data, args=(urls,data_categories))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

C:\Users\babo\AppData\Local\Temp\ipykernel_38180\2929731470.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='F:\python\geckodriver.exe', options=options)



          Dunnes Stores
 


          RTÉ Player
 


          Tesco Ireland - Home Shopping
 


          Uber - Request a ride
 


          WhatsApp Messenger
 


          Google
 


          JD Sports
 


          Just Eat - Food Delivery
 


          H&M - we love fashion
 


          FREE NOW
 


          Instagram
 


          Amazon
 


          TikTok - Videos, Music & LIVE
 


          YouTube: Watch, Listen, Stream
 


          BeReal. Your friends for real.
 


          Google Maps - Transit & Food
 


          CapCut - Video Editor
 


          FotMob - Football Live Scores
 


          Nike: Shop Clothes & Sneakers
 


          Revolut - Mobile Finance
 

